In [4]:
# 📘 1. 라이브러리 불러오기
import pandas as pd
import os

# 📁 2. 엑셀 파일 로드
df = pd.read_csv("../../data/12_04_08_E_CCTV정보.csv", encoding="cp949")

# 🧼 3. 컬럼명 통일 (분석 흐름 유지)
df.rename(columns={
    '소재지도로명주소': '주소',
    'WGS84위도': '위도',
    'WGS84경도': '경도',
    '카메라대수': 'CCTV 수량',
    '설치목적구분': '설치목적'
}, inplace=True)

# ✅ 4. 수치형 변환 & 결측치 제거
df['CCTV 수량'] = pd.to_numeric(df['CCTV 수량'], errors='coerce').fillna(0).astype(int)
df = df.dropna(subset=['위도', '경도'])

# 📦 5. 필요한 열만 저장 (원하면 전체도 가능)
columns_to_save = ['주소', '위도', '경도', 'CCTV 수량', '설치목적']
df_final = df[columns_to_save]

# 💾 6. CSV로 저장
output_path = "../../results/proceeded_cctv.csv"
os.makedirs("../../results", exist_ok=True)
df_final.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 저장 완료: proceeded_cctv.csv")

✅ 저장 완료: proceeded_cctv.csv
